In [ ]:
# pip install flotorch

In [9]:
#import utils
import sys
import os

# Add the parent directory to the Python path
# Assuming utils.py is in a parent directory relative to main_script.py
sys.path.append(os.path.abspath(os.path.join(os.path.dirname("ContentWriter.ipynb"), '../flotorch-rag-notebooks/faiss-example/'))) 

import utils 


In [10]:
# --- Imports from Flotorch ADK and Google ADK ---
from flotorch.adk.agent import FlotorchADKAgent       # Flotorch wrapper for creating an ADK agent
from flotorch.adk.sessions import FlotorchADKSession # Flotorch session service (integrates with Flotorch Gateway sessions)
from google.adk import Runner                         # Runs the agent with a session
from google.genai import types                        # Data types for message content


In [11]:
os.environ["FLOTORCH_API_KEY"] = "sk_HXWY3QOdut4KW0A8FrmnmKu/sRmQxGKUAwm7W8PDDKs=_OTFjMzVkODEtOWYzZi00OTQ0LWJjNjEtZjllY2MyZTA3NGE2_MzJkZDFlNjktOTc0Yy00NTNmLWI0NmUtMWJjMzgzYzc0MmQ2"

In [19]:
# --- Configuration ---
APP_NAME = "flotorch_agent_example"         # Name of the app (used in session tracking)
USER_ID = "flotorch_user_123"               # Unique ID for the user
FLOTORCH_GATEWAY_BASE_URL = "https://gateway.flotorch.cloud"              # Base URL of the Flotorch Gateway
FLOTORCH_API_KEY = os.environ["FLOTORCH_API_KEY"]                      # API key for authentication (to be filled in)
FLOTORCH_AGENT_NAME_TOPIC_CREATOR = "topic-creator"                     # Name of the agent configured in Flotorch
FLOTORCH_AGENT_NAME_CONTENT_WRITER = "content-writer"                     # Name of the agent configured in Flotorch


In [ ]:
# --- Create Flotorch ADK Agent Client for topic-creator---
# This object will fetch the agent definition from the Flotorch Gateway
flotorch_adk_client = FlotorchADKAgent(
    agent_name=FLOTORCH_AGENT_NAME_TOPIC_CREATOR,
    enable_memory=False,                     # Disable memory for this example (can be True to persist chat context for long term)
    base_url=FLOTORCH_GATEWAY_BASE_URL,       
    api_key=FLOTORCH_API_KEY                  
)

# Get the agent object from Flotorch ADK
topic_creator_agent = flotorch_adk_client.get_agent()

flotorch_adk_client_content_writer = FlotorchADKAgent(
    agent_name=FLOTORCH_AGENT_NAME_CONTENT_WRITER,
    enable_memory=False,                     # Disable memory for this example (can be True to persist chat context for long term )
    base_url=FLOTORCH_GATEWAY_BASE_URL,       
    api_key=FLOTORCH_API_KEY                  
)   

content_writer_agent = flotorch_adk_client_content_writer.get_agent()


2025-10-06 21:43:29 - flotorch.sdk.llm - INFO - FlotorchLLM initialized (model_id=flotorch/simple-model-without-caching:latest, base_url=https://gateway.flotorch.cloud)
2025-10-06 21:43:29 - flotorch.adk.llm - INFO - FlotorchADKLLM initialized (model_id=flotorch/simple-model-without-caching:latest, base_url=https://gateway.flotorch.cloud)
2025-10-06 21:43:29 - flotorch.adk.agent - INFO - FlotorchADKAgent initialized (agent_name=topic-creator, memory_enabled=False)


In [21]:
# --- Setup Session Service ---
session_service = FlotorchADKSession(api_key=FLOTORCH_API_KEY, base_url=FLOTORCH_GATEWAY_BASE_URL)

# --- Create Runner ---
# Runner manages the conversation loop between the agent and the user
runner1 = Runner(
    agent=topic_creator_agent,
    app_name=APP_NAME,
    session_service=session_service
)

2025-10-06 21:43:32 - flotorch.sdk.session - INFO - FlotorchSession initialized (base_url=https://gateway.flotorch.cloud)
2025-10-06 21:43:32 - flotorch.adk.sessions - INFO - FlotorchADKSession initialized (base_url=https://gateway.flotorch.cloud)


In [ ]:
runner2 = Runner(
    agent=content_writer_agent,
    app_name=APP_NAME,
    session_service=session_service
)


In [ ]:
# --- Function to send a single user message and get the final AI response ---
def run_single_turn(query, session_id, runner,user_id):
    """Send a message to the Flotorch agent and return the final response text."""
    # Create message content object (role=user, with text parts)
    content = types.Content(role="user", parts=[types.Part(text=query)])
    
    # Run the agent for this message in the given session
    events = runner.run(
        user_id=user_id, 
        session_id=session_id, 
        new_message=content
    )

    # Loop through the events to find the final response from the agent
    for event in events:
        if event.is_final_response():
            if event.content and event.content.parts:
                return event.content.parts[0].text  # Return the text from the first part
    
    return "Sorry, I couldn't process that."  # Fallback if no valid response


In [ ]:
# --- Async function to start a chat session and send a message ---
async def chat_with_ui_agent(message):
    """Start a new session with the agent and send a single message."""
    # Create a new session for the user via Flotorch Gateway
    session = await runner1.session_service.create_session(
        app_name=APP_NAME,
        user_id=USER_ID,
    )
    
    # Send the message and get the response
    response_topic_creator = run_single_turn(message, session.id, runner1,USER_ID)
    
    # Print the conversation
    print(f"You: {message}")
    print(f"Flotorch AI: {response_topic_creator}")
    
    return response_topic_creator

    session = await runner2.session_service.create_session(
        app_name=APP_NAME,
        user_id=USER_ID,
    )

    response_content_writer = run_single_turn(response_topic_creator, session.id, runner2,USER_ID)

    return response_content_writer

In [24]:
# --- Entry point ---
# Changed to call the synchronous function directly
if __name__ == "__main__":
    await(chat_with_ui_agent("Scan the following paragraph: In Jurassic World Rebirth, the world's dinosaurs live around the equator, which provides the last viable climate for them to survive. A team travels to a former island research facility where the three largest prehistoric animals reside, with the goal of extracting samples that are vital for a heart disease treatment. The team also rescues a shipwrecked family, and both groups struggle to survive after becoming stranded on the island."))


2025-10-06 21:43:44 - flotorch.sdk.session - INFO - Session Create (uid: b9b69dcb-89ec-4dac-8438-57a2f61f3f6b) - {'app_name': 'flotorch_agent_example', 'user_id': 'flotorch_user_123'}
2025-10-06 21:43:44 - flotorch.sdk.session - INFO - Session Created (uid: b9b69dcb-89ec-4dac-8438-57a2f61f3f6b) - {'app_name': 'flotorch_agent_example', 'user_id': 'flotorch_user_123'}
2025-10-06 21:43:44 - flotorch.sdk.session - INFO - Session Get (uid: b9b69dcb-89ec-4dac-8438-57a2f61f3f6b) - {'after_timestamp': None, 'num_recent_events': None}
2025-10-06 21:43:44 - flotorch.sdk.session - INFO - Session Retrieved (uid: b9b69dcb-89ec-4dac-8438-57a2f61f3f6b) - {'events_count': 0}
2025-10-06 21:43:44 - flotorch.sdk.session - INFO - Session Get (uid: b9b69dcb-89ec-4dac-8438-57a2f61f3f6b) - {'after_timestamp': None, 'num_recent_events': None}
2025-10-06 21:43:45 - flotorch.sdk.session - INFO - Session Retrieved (uid: b9b69dcb-89ec-4dac-8438-57a2f61f3f6b) - {'events_count': 0}
2025-10-06 21:43:45 - flotorch.sd